In [4]:
import pandas as pd
import numpy as np

import os
import glob

import re

# pd.options.display.max_columns = None

## Load Data

In [5]:
mydir = r"D:\PURWADHIKA\FINAL PROJECT\ecommerce-analysis\data\raw"

file_list = glob.glob(mydir + "\*.csv")
file_list 

['D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_customers_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_geolocation_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_orders_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_items_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_payments_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_reviews_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_products_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_sellers_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\product_category_name_translation.csv']

In [6]:
def efn(text):
    file = text.split('\\')[-1].replace('.csv','')
    return file

In [7]:
dfs = {f'{efn(file)}_df': pd.read_csv(file) for file in file_list}

In [8]:
for n in dfs.keys():
    print(n)

olist_customers_dataset_df
olist_geolocation_dataset_df
olist_orders_dataset_df
olist_order_items_dataset_df
olist_order_payments_dataset_df
olist_order_reviews_dataset_df
olist_products_dataset_df
olist_sellers_dataset_df
product_category_name_translation_df


## Data Understanding

In [9]:
dataInfo = []

for n in dfs.keys():
    dataInfo.append({
        'dataset_name' : n,
        'shape' : dfs[n].shape,
        'columns' : list(dfs[n].columns),
        'duplicate' : dfs[n].duplicated().sum(),
        'null' : dfs[n].isna().sum().sum(),
        'null_columns' : dfs[n].columns[dfs[n].isna().any()].tolist()
    })

pd.DataFrame(dataInfo)


,dataset_name,shape,columns,duplicate,null,null_columns
0,olist_customers_dataset_df,"(99441, 5)","[customer_id, customer_unique_id, customer_zip...",0,0,[]
1,olist_geolocation_dataset_df,"(1000163, 5)","[geolocation_zip_code_prefix, geolocation_lat,...",261831,0,[]
2,olist_orders_dataset_df,"(99441, 8)","[order_id, customer_id, order_status, order_pu...",0,4908,"[order_approved_at, order_delivered_carrier_da..."
3,olist_order_items_dataset_df,"(112650, 7)","[order_id, order_item_id, product_id, seller_i...",0,0,[]
4,olist_order_payments_dataset_df,"(103886, 5)","[order_id, payment_sequential, payment_type, p...",0,0,[]
5,olist_order_reviews_dataset_df,"(99224, 7)","[review_id, order_id, review_score, review_com...",0,145903,"[review_comment_title, review_comment_message]"
6,olist_products_dataset_df,"(32951, 9)","[product_id, product_category_name, product_na...",0,2448,"[product_category_name, product_name_lenght, p..."
7,olist_sellers_dataset_df,"(3095, 4)","[seller_id, seller_zip_code_prefix, seller_cit...",0,0,[]
8,product_category_name_translation_df,"(71, 2)","[product_category_name, product_category_name_...",0,0,[]


In [10]:
# skimming function

def skim(data):
    summary = pd.DataFrame({
                    'column': data.columns.values,
                    'type': data.dtypes.values,
                    'n_unique': data.nunique().values,
                    'n_missing': data.isna().sum().reset_index()[0],
                    'pct_missing': round(data.isna().sum().reset_index()[0]/len(data)*100, 2),
                    'min': data.dropna().min().values,
                    'max': data.dropna().max().values
                })
    return summary

**olist_customers_dataset_df**

In [11]:
skim(dfs['olist_customers_dataset_df'])

,column,type,n_unique,n_missing,pct_missing,min,max
0,customer_id,object,99441,0,0.0,00012a2ce6f8dcda20d059ce98491703,ffffe8b65bbe3087b653a978c870db99
1,customer_unique_id,object,96096,0,0.0,0000366f3b9a7992bf8c76cfdf3221e2,ffffd2657e2aad2907e67c3e9daecbeb
2,customer_zip_code_prefix,int64,14994,0,0.0,1003,99990
3,customer_city,object,4119,0,0.0,abadia dos dourados,zortea
4,customer_state,object,27,0,0.0,AC,TO


**olist_geolocation_dataset_df**

**olist_orders_dataset_df**

**olist_order_items_dataset_df**

**olist_order_payments_dataset_df**

**olist_order_reviews_dataset_df**

**olist_products_dataset_df**

In [12]:
skim(dfs['olist_products_dataset_df'])

,column,type,n_unique,n_missing,pct_missing,min,max
0,product_id,object,32951,0,0.00,00066f42aeeb9f3007548bb9d3f33c38,fffe9eeff12fcbd74a2f2b007dde0c58
1,product_category_name,object,73,610,1.85,agro_industria_e_comercio,utilidades_domesticas
2,product_name_lenght,float64,66,610,1.85,5.0,76.0
3,product_description_lenght,float64,2960,610,1.85,4.0,3992.0
4,product_photos_qty,float64,19,610,1.85,1.0,20.0
5,product_weight_g,float64,2204,2,0.01,0.0,40425.0
6,product_length_cm,float64,99,2,0.01,7.0,105.0
7,product_height_cm,float64,102,2,0.01,2.0,105.0
8,product_width_cm,float64,95,2,0.01,6.0,118.0


**olist_sellers_dataset_df**

**product_category_name_translation_df**